In [2]:
!pip install delu rtdl_revisiting_models

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://nexus.isb/repository/pypiorg-pypi-proxy/simple


In [8]:
!pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://nexus.isb/repository/pypiorg-pypi-proxy/simple
     |████████████████████████████████| 7.0 MB 11.8 MB/s eta 0:00:01�███              | 3.9 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 22.7 MB/s eta 0:00:01


In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import delu
#import rtdl
from rtdl_revisiting_models import FTTransformer

from tqdm.std import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from functools import partial
import sklearn.model_selection
import sklearn.preprocessing
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import roc_auc_score

import otp.stable.connections as otpc

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 500)

username = ****
password = ****

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# df = otpc.load_oracle('''
# select *
# from iar_transformer_model_data_prefinal_ver4
# ''',username, password)

In [3]:
df = pd.read_csv('dataset_with_transf_prob.csv')
df.head()

,application_id,product_grp,agr_lims,used_score,personal_income,payment,ltb,circle,paymax,cnt_all_apps_last7days,cnt_all_apps_last30days,cnt_all_apps_last90days,cnt_refused_apps_last7days,cnt_refused_apps_last30days,cnt_refused_apps_last90days,online_flg,bureau_hist_due,appnumberofappslm,count_bureau_credits_less_730_days,client_segment,target,rn,cnt_open_agr_7,cnt_open_agr_30,cnt_open_agr_90,transf_prob
0,EM-1291754,cash,55000,687,38000,2500,5,5,4,6,6,7,0,0,0,1,4817,-1,-1,OTP,0,1,0,0,0,0.160092
1,EM-1381870,cash,218771,666,95000,6281,5,5,3,3,3,3,0,0,0,1,3973,-1,-1,OTP,0,1,0,0,0,0.061289
2,EM-1382144,cash,106233,671,50000,9340,5,5,3,10,10,10,4,4,4,1,3698,-1,-1,OTP,0,1,0,0,0,0.826771
3,EM-1382832,cash,130671,678,30000,3764,5,5,3,6,6,6,0,0,0,1,4374,-1,-1,OTP,1,1,0,0,0,0.047599
4,EM-1392324,cash,190000,633,35000,5455,5,1,2,4,4,4,1,1,1,1,5729,-1,-1,OTP,1,1,0,0,0,0.744719


In [5]:
df.columns = list(map(str.lower, df.columns))

In [6]:
df = df.set_index('application_id')
df.head()

,product_grp,agr_lims,used_score,personal_income,payment,ltb,circle,paymax,cnt_all_apps_last7days,cnt_all_apps_last30days,cnt_all_apps_last90days,cnt_refused_apps_last7days,cnt_refused_apps_last30days,cnt_refused_apps_last90days,online_flg,bureau_hist_due,appnumberofappslm,count_bureau_credits_less_730_days,client_segment,target,rn,cnt_open_agr_7,cnt_open_agr_30,cnt_open_agr_90,transf_prob
application_id,,,,,,,,,,,,,,,,,,,,,,,,,
EM-1291754,cash,55000,687,38000,2500,5,5,4,6,6,7,0,0,0,1,4817,-1,-1,OTP,0,1,0,0,0,0.160092
EM-1381870,cash,218771,666,95000,6281,5,5,3,3,3,3,0,0,0,1,3973,-1,-1,OTP,0,1,0,0,0,0.061289
EM-1382144,cash,106233,671,50000,9340,5,5,3,10,10,10,4,4,4,1,3698,-1,-1,OTP,0,1,0,0,0,0.826771
EM-1382832,cash,130671,678,30000,3764,5,5,3,6,6,6,0,0,0,1,4374,-1,-1,OTP,1,1,0,0,0,0.047599
EM-1392324,cash,190000,633,35000,5455,5,1,2,4,4,4,1,1,1,1,5729,-1,-1,OTP,1,1,0,0,0,0.744719


In [ ]:
del df['paymax'], df['rn']

In [7]:
y = df[['target']]
X = df.drop(columns=['target'])

### preparing dataset for Transformer model:

In [8]:
X_cat = X.select_dtypes(include=['object'])

X_cat = X_cat.apply(LabelEncoder().fit_transform)

X_float = X.select_dtypes(include=['float'])
X_int_float = X.select_dtypes(include=['int64']).astype(np.float32)
X_float = pd.concat([X_float, X_int_float], axis=1)
X_float = X_float.astype(np.float32)

float_features_cnt = X_float.shape[1]
cat_features_card = [X_cat[x].nunique() for x in X_cat.columns]

y = y.astype('float')

In [9]:
X_float = X_float.to_numpy()
X_cat = X_cat.to_numpy()
y = y.to_numpy()
y = y[:, 0]

In [10]:
idx = np.arange(len(y))
train_val_idx, test_idx = sklearn.model_selection.train_test_split(
    idx, train_size=0.8, random_state=1
)
train_idx, val_idx = sklearn.model_selection.train_test_split(
    train_val_idx, train_size=0.8, random_state=1
)

In [11]:
data = {
    'train' : {'x_float':X_float[train_idx], 'x_cat':X_cat[train_idx], 'y':y[train_idx]},
    'val' : {'x_float':X_float[val_idx], 'x_cat':X_cat[val_idx], 'y':y[val_idx]},
    'test' : {'x_float':X_float[test_idx], 'x_cat':X_cat[test_idx], 'y':y[test_idx]}
}

In [12]:
preprocessing = sklearn.preprocessing.StandardScaler().fit(data['train']['x_float'])

for object in data:
    data[object]['x_float'] = preprocessing.transform(data[object]['x_float'])

In [13]:
tensors = {
    object: {k: torch.as_tensor(v, device=device) for k, v in data[object].items()} for object in data
}

In [14]:
tensors

{'train': {'x_float': tensor([[ -0.8045,  -0.5096,  -0.0873,  ...,  -0.0689,  -0.1338,  -0.1899],
          [  0.5592,  -0.4467,   1.0236,  ...,  -0.0689,  -0.1338,  -0.1899],
          [ -0.7894,  -0.3767, -10.4708,  ...,  -0.0689,  -0.1338,  -0.1899],
          ...,
          [ -0.6488,  -0.2999,   1.0082,  ...,  -0.0689,  -0.1338,  -0.1899],
          [  0.0850,   1.6896,  -0.2878,  ...,  -0.0689,  -0.1338,  -0.1899],
          [ -0.9221,   0.3415,  -1.3524,  ...,  -0.0689,  -0.1338,  -0.1899]]),
  'x_cat': tensor([[1, 0],
          [1, 2],
          [0, 2],
          ...,
          [1, 2],
          [0, 2],
          [1, 1]]),
  'y': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)},
 'val': {'x_float': tensor([[-1.1590, -0.4735,  0.4990,  ..., -0.0689, -0.1338, -0.1899],
          [-0.6036, -0.1906, -0.5347,  ..., -0.0689, -0.1338, -0.1899],
          [-1.2120, -0.6474,  0.7459,  ..., -0.0689, -0.1338, -0.1899],
          ...,
          [ 0.9731, -0.0501,  0.9619,  ...,

In [15]:
sum(sum([tensors['test']['y'].cpu()==1]))

tensor(7910)

In [16]:
tensors['train']['y'].shape

torch.Size([192000])

### Transformer model

In [17]:
model = FTTransformer(
    n_cont_features=float_features_cnt,
    cat_cardinalities=cat_features_card,
    d_out=1,
    **FTTransformer.get_default_kwargs(),
).to(device)
optimizer = model.make_default_optimizer()

In [18]:
def apply_model(batch):
    return model(batch['x_float'], batch['x_cat']).squeeze(-1)

In [19]:
pos_weight = 5 # = 10 if name == 'fraud' else 4
loss_func = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(pos_weight))

In [20]:
def evaluate(part, eval_batch_size):
    model.eval()
    with torch.inference_mode():
        y_pred = (
            torch.cat(
                [
                    apply_model(batch) for batch in delu.iter_batches(tensors[part], eval_batch_size)
                ]
            )
            .cpu()
        )
        y_true = tensors[part]['y'].cpu().numpy()

        y_pred = torch.sigmoid(y_pred).numpy()
        test_y_pred = y_pred
        y_pred = np.where(y_pred>=0.85, 1, 0)
        #1 if y_pred>=0.5 else 0
        #score = sklearn.metrics.accuracy_score(y_true, y_pred)
        score = sklearn.metrics.roc_auc_score(y_true, test_y_pred)
        precision = sklearn.metrics.precision_score(y_true, y_pred)
        recall = sklearn.metrics.recall_score(y_true, y_pred)
    return score, precision, recall, test_y_pred

In [21]:
n_epochs = 50
patience = 100

eval_batch_size = 512
batch_size = 512
epoch_size = math.ceil(len(train_idx) / batch_size)
timer = delu.tools.Timer()
early_stopping = delu.tools.EarlyStopping(patience, mode='max')

print(f"Device: {device}")
print("-" * 88 + "\n")
timer.run()
for epoch in range(n_epochs):
    for batch in tqdm(
        delu.iter_batches(tensors['train'], batch_size, shuffle=True),
        desc=f"Epoch {epoch}",
        total=epoch_size,
    ):
        model.train()
        optimizer.zero_grad()
        loss = loss_func(apply_model(batch), batch['y'])
        loss.backward()
        optimizer.step()

    val_score, val_precision, val_recall, prob_val = evaluate('val', eval_batch_size)
    print(f"Rocauc: (val) {val_score:.4f} [time] {timer}")
    print(f"Precision: (val) {val_precision:.4f} [time] {timer}")
    print(f"Recall: (val) {val_recall:.4f} [time] {timer}")

    early_stopping.update(val_score)
    if early_stopping.should_stop():
        break

    print()

Device: cpu
----------------------------------------------------------------------------------------



Epoch 0: 100%|██████████| 375/375 [04:01<00:00,  1.55it/s]


Rocauc: (val) 0.7863 [time] 0:04:17.758283
Precision: (val) 0.7100 [time] 0:04:17.758561
Recall: (val) 0.0800 [time] 0:04:17.758577



Epoch 1: 100%|██████████| 375/375 [04:02<00:00,  1.55it/s]


Rocauc: (val) 0.7907 [time] 0:08:35.773970
Precision: (val) 0.7113 [time] 0:08:35.774916
Recall: (val) 0.0915 [time] 0:08:35.774945



...

Rocauc: (val) 0.8031 [time] 8:01:56.284205
Precision: (val) 0.6661 [time] 8:01:56.284488
Recall: (val) 0.1767 [time] 8:01:56.284503



Epoch 48: 100%|██████████| 375/375 [13:47<00:00,  2.21s/it]


Rocauc: (val) 0.8042 [time] 8:16:53.961584
Precision: (val) 0.6727 [time] 8:16:53.961851
Recall: (val) 0.1648 [time] 8:16:53.961866



Epoch 49: 100%|██████████| 375/375 [14:04<00:00,  2.25s/it]


Rocauc: (val) 0.8047 [time] 8:32:09.676193
Precision: (val) 0.6393 [time] 8:32:09.676457
Recall: (val) 0.1927 [time] 8:32:09.676472



In [22]:
test_score, test_precision, test_recall, prob_test= evaluate('test', eval_batch_size)
print(f"Rocauc: (test) {test_score:.4f}")
print(f"Precision: (test) {test_precision:.4f}")
print(f"Recall: (test) {test_recall:.4f}")
print(prob_test[tensors['test']['y'].cpu()==1])
print(tensors['test']['y'][(tensors['test']['y']==1)])

Rocauc: (test) 0.8061
Precision: (test) 0.6400
Recall: (test) 0.1989
[0.139012   0.7236898  0.92695594 ... 0.86959255 0.21760282 0.8434043 ]
tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64)


In [23]:
sklearn.metrics.confusion_matrix(tensors['test']['y'].cpu().numpy(), np.where(prob_test>=0.85, 1, 0))

array([[51205,   885],
       [ 6337,  1573]])

In [27]:
# _, _, _, prob_train= evaluate('train', eval_batch_size)
# val_score, val_precision, val_recall, prob_val = evaluate('val', eval_batch_size)

In [29]:
# prob = np.concatenate([prob_train, prob_val, prob_test])
# df['transf_prob'] = prob.tolist()

In [31]:
# df.to_csv('dataset_with_transf_prob.csv')